In [11]:
# ! pip install pypdf
# ! pip install streamlit
# ! pip install ollama
# ! pip install langchain_community
# ! pip install langchain-text-splitters
# ! pip install python-docx
# ! pip install faiss-gpu
# ! pip install "numpy<2"
# ! pip install torch
# ! pip install sentence-transformers

In [12]:
! rm app.py

In [13]:
%%writefile app.py

import os
import re
import html
import json
import time
import faiss
import torch
import tempfile
import numpy as np
import streamlit as st
import logging
from pathlib import Path
from docx import Document
from ollama import Client
from docx.table import Table as _Table
from docx.text.paragraph import Paragraph as _Paragraph
from sentence_transformers import SentenceTransformer, util
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

Writing app.py


In [14]:
%%writefile -a app.py

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("app.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

Appending to app.py


In [15]:
%%writefile -a app.py

ollama = Client()

st.set_page_config(
    page_title="Simple RAG with Ollama & LaBSE (FAISS)",
    page_icon="😬",
    layout="wide"
)

CHUNK_SIZE = 1500
CHUNK_OVERLAP = 200
EMBEDDING_MODEL_NAME = "sentence-transformers/LaBSE"
LLM_MODEL = "gemma2:2b"
FINGLISH_MAP_FILE = "sample_finglish_map.json"
TOP_K = 5

FIXED_DISCLAIMER_FA = "Disclaimer: این پاسخ صرفاً جنبه اطلاع‌رسانی دارد و جایگزین مشاوره تخصصی حقوقی/مالی نیست."
TLDR_WORD_COUNT_THRESHOLD = 300

DEFAULT_PROMPT_SYSTEM = (
    "شما یک دستیار هوشمند هستید. شما **فقط** باید بر اساس متنی که به شما داده می‌شود (Context) پاسخ دهید. "
    "اگر پاسخ به هیچ وجه در متن یافت نشد، باید با عبارت دقیق **«پاسخ در متن‌های ارائه شده وجود ندارد.»** به زبان فارسی پاسخ را شروع کنید. "
    "از هرگونه گمانه‌زنی، دانش قبلی یا پاسخ‌های عمومی خودداری کنید."
    )


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
st.sidebar.info(f"Running on device: {DEVICE}")

Appending to app.py


In [16]:
%%writefile -a app.py

ollama = Client()

st.set_page_config(
    page_title="Simple RAG with Ollama & LaBSE (FAISS)",
    page_icon="😬",
    layout="wide"
)

CHUNK_SIZE = 1500
CHUNK_OVERLAP = 200
EMBEDDING_MODEL_NAME = "sentence-transformers/LaBSE"
LLM_MODEL = "gemma2:2b"
FINGLISH_MAP_FILE = "sample_finglish_map.json"
TOP_K = 5

FIXED_DISCLAIMER_FA = "Disclaimer: این پاسخ صرفاً جنبه اطلاع‌رسانی دارد و جایگزین مشاوره تخصصی حقوقی/مالی نیست."
TLDR_WORD_COUNT_THRESHOLD = 300

DEFAULT_PROMPT_SYSTEM = (
    "شما یک دستیار هوشمند هستید. شما **فقط** باید بر اساس متنی که به شما داده می‌شود (Context) پاسخ دهید. "
    "اگر پاسخ به هیچ وجه در متن یافت نشد، باید با عبارت دقیق **«پاسخ در متن‌های ارائه شده وجود ندارد.»** به زبان فارسی پاسخ را شروع کنید. "
    "از هرگونه گمانه‌زنی، دانش قبلی یا پاسخ‌های عمومی خودداری کنید."
    )


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
st.sidebar.info(f"Running on device: {DEVICE}")
logger.info(f"Application started. Running on device: {DEVICE}")

def load_pdf(file_path):
    loader = PyPDFLoader(str(file_path))
    docs = loader.load()
    text = "\n".join(page.page_content for page in docs)
    return text


class DocParser:
    def __init__(self, file_path, include_empty=False):
        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.info(f"Initializing DocParser for: {file_path}")
        self.doc = Document(file_path)
        self.include_empty = include_empty

    def _table_to_markdown(self, table):
        markdown = []
        for i, row in enumerate(table.rows):
            row_data = [cell.text.strip() for cell in row.cells]
            markdown.append("| " + " | ".join(row_data) + " |")
            if i == 0:
                markdown.append("| " + " | ".join(["---"] * len(row_data)) + " |")
        return "\n".join(markdown)

    def _iter_blocks(self):
        for child in self.doc.element.body:
            if child.tag.endswith('p'):
                yield _Paragraph(child, self.doc)
            elif child.tag.endswith('tbl'):
                yield _Table(child, self.doc)

    def content_extract(self):
        self.logger.info("Starting content extraction...")
        content = []
        try:
            for block in self._iter_blocks():
                if isinstance(block, _Paragraph):
                    text = block.text.strip()
                    if text or self.include_empty:
                        content.append(text)
                elif isinstance(block, _Table):
                    content.append(self._table_to_markdown(block))
            self.logger.info("Content extraction successful.")
            return "\n\n".join(content)
        except Exception as e:
            self.logger.error(f"Error during DocParser content_extract: {e}", exc_info=True)
            return ""


def load_json_mapping(file_path):
    try:
        map_path = Path(file_path)
        if not map_path.exists():
            logger.warning(f"Finglish map file not found: {file_path}. Finglish embedding will be disabled.")
            st.warning(f"⚠️ The Finglish embedding does not exist ({file_path}). Finglish embedding will be disabled")
            return {}

        with open(map_path, 'r', encoding='utf-8') as f:
            mapping = json.load(f)
            if isinstance(mapping, dict):
                logger.info(f"Finglish map loaded with {len(mapping)} entries from {file_path}.")
                st.success(f"✅ Finglish embedding loaded with length {len(mapping)} from {file_path}.")
                return mapping
            else:
                logger.error(f"File {file_path} is not a valid dictionary.")
                st.error(f"❌ The file {file_path} does not include a valid dictionary.")
                return {}

    except json.JSONDecodeError as e:
        logger.error(f"JSON Decode Error for {file_path}: {e}", exc_info=True)
        st.error(f"❌ The file {file_path} faced with JSON Decode Error. Make sure of the structure!")
        return {}
    except Exception as e:
        logger.error(f"Unknown error loading Finglish embedding: {e}", exc_info=True)
        st.error(f"❌ Unknown error while loading the Finglish embedding: {e}")
        return {}


def map_finglish_to_persian(query_text, mapping_dict):
    processed_query = query_text
    for finglish, persian in mapping_dict.items():
        pattern = r'\b' + re.escape(finglish) + r'\b'
        processed_query = re.sub(pattern, persian, processed_query, flags=re.IGNORECASE)
    
    if processed_query != query_text:
        logger.info("Finglish-to-Persian mapping applied to query.")
    return processed_query


def filter_persian_english(text):
    """
    Removes any characters that are not in the English, Persian,
    number, or common punctuation/whitespace Unicode ranges.
    """
    pattern = r'[^\x00-\x7F\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF]'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

Appending to app.py


In [17]:
%%writefile -a app.py

class SimpleRAGSystem:

    def __init__(self):
        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.info("Initializing SimpleRAGSystem...")
        with st.spinner(f"Loading embedding model '{EMBEDDING_MODEL_NAME}'..."):
            try:
                self.logger.info(f"Loading embedding model: {EMBEDDING_MODEL_NAME}")
                self.model_ST = SentenceTransformer(EMBEDDING_MODEL_NAME, device=DEVICE)
                self.embedding_dim = self.model_ST.get_sentence_embedding_dimension()
                self.logger.info(f"Embedding model loaded. Dimension: {self.embedding_dim}")
                st.success(f"✅ Embedding model loaded. Dimension: {self.embedding_dim}")
            except Exception as e:
                self.logger.error(f"Error loading embedding model: {e}", exc_info=True)
                st.error(f"Error loading embedding model: {e}")
                st.stop()


    def extract_text_from_file(self, uploaded_file):
        self.logger.info(f"Attempting to extract text from file: {uploaded_file.name}")
        try:
            with tempfile.NamedTemporaryFile(delete=False, suffix=Path(uploaded_file.name).suffix) as tmp_file:
                tmp_file.write(uploaded_file.getvalue())
                tmp_file_path = tmp_file.name

            text = ""
            file_suffix = Path(tmp_file_path).suffix
            self.logger.info(f"Processing temporary file: {tmp_file_path} (Type: {file_suffix})")

            if file_suffix == ".pdf":
                text = load_pdf(tmp_file_path)
            elif file_suffix == ".docx":
                doc_parser = DocParser(tmp_file_path)
                text = doc_parser.content_extract()
            else:
                self.logger.warning(f"Unsupported file type: {file_suffix}")
                st.error(f"Unsupported file type: {file_suffix}")
                return None

            os.remove(tmp_file_path)
            self.logger.info(f"Successfully extracted text from {uploaded_file.name}")
            return text
        except Exception as e:
            self.logger.error(f"Failed to read file: {uploaded_file.name}", exc_info=True)
            st.error(f"Error reading file: {str(e)}")
            return None


    def create_chunks(self, text, source_name):
        self.logger.info(f"Creating chunks for: {source_name}")
        try:
            splitter = RecursiveCharacterTextSplitter(
                chunk_size=CHUNK_SIZE,
                chunk_overlap=CHUNK_OVERLAP
            )
            text_chunks = splitter.split_text(text)

            chunks = []
            for i, chunk_text in enumerate(text_chunks):
                if chunk_text.strip():
                    chunks.append({
                        'id': f"{source_name}_{i}",
                        'text': chunk_text.strip(),
                        'source': source_name,
                        'chunk_number': i
                    })
            self.logger.info(f"Created {len(chunks)} chunks for {source_name}")
            return chunks
        except Exception as e:
            self.logger.error(f"Error creating chunks for {source_name}: {e}", exc_info=True)
            st.error(f"Error creating chunks: {e}")
            return []


    def generate_embeddings(self, texts):
        self.logger.info(f"Generating embeddings for {len(texts)} text chunks...")
        try:
            embeddings = self.model_ST.encode(
                texts,
                show_progress_bar=True,
                convert_to_tensor=False,
                device=DEVICE
            )
            self.logger.info("Embedding generation successful.")
            return np.ascontiguousarray(embeddings)
        except Exception as e:
            self.logger.error(f"Error generating embeddings: {e}", exc_info=True)
            st.error(f"Error generating embeddings: {str(e)}")
            return None


    def create_faiss_index(self, new_embeddings, current_index):
        self.logger.info("Creating/updating FAISS index...")
        try:
            if current_index is None:
                index = faiss.IndexFlatL2(self.embedding_dim)
                index.add(new_embeddings.astype('float32'))
                self.logger.info(f"New FAISS index created with {index.ntotal} vectors.")
                st.success(f"New FAISS index created with {index.ntotal} vectors.")
                return index
            else:
                current_index.add(new_embeddings.astype('float32'))
                self.logger.info(f"Appended {len(new_embeddings)} vectors. Total: {current_index.ntotal}")
                st.success(f"Appended {len(new_embeddings)} vectors. Total: {current_index.ntotal}")
                return current_index
        except Exception as e:
            self.logger.error(f"Error creating/updating FAISS index: {e}", exc_info=True)
            st.error(f"Error creating/updating FAISS index: {str(e)}")
            return None


    def search_documents(self, query, text_chunks, faiss_index, top_k=TOP_K):
        self.logger.info(f"Searching for query: '{query[:50]}...'")
        if not text_chunks or faiss_index is None:
            self.logger.warning("Search attempted, but no documents processed yet.")
            st.warning("No documents have been processed yet.")
            return None

        try:
            query_embedding = self.model_ST.encode(
                query,
                convert_to_tensor=False,
                device=DEVICE
            ).astype('float32').reshape(1, -1)

            distances, indices = faiss_index.search(query_embedding, top_k)
            self.logger.info(f"FAISS search completed. Found {len(indices[0])} potential matches.")

            documents = []
            metadatas = []

            for i, idx in enumerate(indices[0]):
                if idx == -1:
                    continue
                chunk = text_chunks[idx]
                score = distances[0][i]
                documents.append(chunk['text'])
                metadatas.append({
                    'source': chunk['source'],
                    'chunk_number': chunk['chunk_number'],
                    'score': score
                })
            
            self.logger.info(f"Returning {len(documents)} valid documents.")
            return {'documents': [documents], 'metadatas': [metadatas]}

        except Exception as e:
            self.logger.error(f"Error searching documents: {e}", exc_info=True)
            st.error(f"Error searching documents: {str(e)}")
            return None


    def generate_answer(self, query, context, chat_history):
        self.logger.info(f"Generating answer with LLM ({LLM_MODEL}) for query: '{query[:50]}...'")
        try:
            history_text = ""
            if chat_history:
                for i, turn in enumerate(chat_history[-5:], start=1):
                    history_text += f"پرسش {i}: {turn['query']}\nپاسخ {i}: {turn['answer']}\n\n"

            prompt = f"""{DEFAULT_PROMPT_SYSTEM}

    **سوابق گفتگو تا کنون:**
    {history_text}

    **دستورالعمل:**
    - بر اساس متن‌ها (Context) و تاریخچه گفتگو، به پرسش جدید پاسخ بده.
    - پاسخ باید به فارسی روان و دقیق باشد.
    - اگر پاسخ در تاریخچه آمده است، از آن استفاده کن ولی آن را خلاصه و بازنویسی کن.

    Context:
    {context}

    پرسش جدید: {query}

    پاسخ:"""

            response = ollama.generate(
                model=LLM_MODEL,
                prompt=prompt,
                options={
                    'temperature': 0.7,
                    'max_tokens': 1000
                }
            )
            self.logger.info("LLM generation successful.")
            return response['response']

        except Exception as e:
            self.logger.error(f"Error generating answer with Ollama: {e}", exc_info=True)
            st.error(f"Error generating answer: {str(e)}")
            return None

    def summarize_text(self, text_to_summarize):
        self.logger.info(f"Generating summary with LLM ({LLM_MODEL})...")
        try:
            prompt = f"""شما یک دستیار خلاصه‌ساز هستید. متن زیر به زبان فارسی است. آن را در یک یا دو جمله روان و دقیق خلاصه کنید:

            متن:
            {text_to_summarize}

            خلاصه:"""

            response = ollama.generate(
                model=LLM_MODEL,
                prompt=prompt,
                options={
                    'temperature': 0.3,
                    'max_tokens': 150
                }
            )
            self.logger.info("Summary generation successful.")
            return response['response'].strip()
        except Exception as e:
            self.logger.error(f"Error generating summary with Ollama: {e}", exc_info=True)
            st.error(f"Error generating summary: {str(e)}")
            return None

Appending to app.py


In [18]:
%%writefile -a app.py

def main():
    logger.info("Starting main() function. Page is loading.")
    st.title("😬 RAG System with Ollama & LaBSE (FAISS)")
    st.markdown("Upload PDF or DOCX documents and ask questions in Farsi or English!")

    if 'rag_system' not in st.session_state:
        logger.info("Initializing 'rag_system' in session state.")
        st.session_state.rag_system = SimpleRAGSystem()

    if 'finglish_map' not in st.session_state:
        logger.info("Loading 'finglish_map' into session state.")
        st.session_state.finglish_map = load_json_mapping(FINGLISH_MAP_FILE)

    if 'processed_files' not in st.session_state:
        st.session_state.processed_files = set()

    if 'text_chunks' not in st.session_state:
        st.session_state.text_chunks = []

    if 'faiss_index' not in st.session_state:
        st.session_state.faiss_index = None

    if 'chat_history' not in st.session_state:
        st.session_state.chat_history = []

    MAX_FILES = 5

    with st.sidebar:
        st.header("📁 Document Upload")

        current_file_count = len(st.session_state.processed_files)
        remaining_slots = MAX_FILES - current_file_count

        st.caption(f"File limit: {current_file_count} / {MAX_FILES} processed.")

        if remaining_slots > 0:
            uploaded_files = st.file_uploader(
                f"Choose PDF or DOCX files (up to {remaining_slots} more)",
                type=["pdf", "docx"],
                help=f"You can upload a total of {MAX_FILES} documents. You have {remaining_slots} slot(s) left.",
                accept_multiple_files=True
            )

            if uploaded_files:
                if len(uploaded_files) > remaining_slots:
                    logger.warning(f"Upload failed: Too many files. Tried {len(uploaded_files)}, only {remaining_slots} slots left.")
                    st.error(
                        f"Upload failed. You tried to upload {len(uploaded_files)} file(s), but you only have {remaining_slots} slot(s) left. Please select fewer files.")
                else:
                    needs_rerun = False
                    start_chunk_idx = len(st.session_state.text_chunks)

                    for uploaded_file in uploaded_files:
                        file_name = uploaded_file.name

                        if file_name not in st.session_state.processed_files:
                            logger.info(f"Processing new file: {file_name}")
                            with st.spinner(f"Processing {file_name}..."):

                                text = st.session_state.rag_system.extract_text_from_file(uploaded_file)

                                if text:
                                    new_chunks = st.session_state.rag_system.create_chunks(text, file_name)
                                    st.info(f"Created {len(new_chunks)} chunks from {file_name}.")

                                    if new_chunks:
                                        new_texts = [chunk['text'] for chunk in new_chunks]
                                        new_embeddings = st.session_state.rag_system.generate_embeddings(new_texts)

                                        if new_embeddings is not None:
                                            st.session_state.text_chunks.extend(new_chunks)
                                            st.session_state.faiss_index = st.session_state.rag_system.create_faiss_index(
                                                new_embeddings,
                                                st.session_state.faiss_index
                                            )
                                            st.session_state.processed_files.add(file_name)
                                            logger.info(f"Successfully processed and indexed {file_name}.")
                                            st.success(f"✅ Successfully processed {file_name}")
                                            needs_rerun = True
                                        else:
                                            logger.error(f"Failed to generate embeddings for {file_name}. Skipping file.")
                                            st.error(f"❌ Failed to generate embeddings for {file_name}")
                                    else:
                                        logger.error(f"No text chunks created for {file_name}. Skipping file.")
                                        st.error(f"❌ No text chunks created for {file_name}")
                                else:
                                    logger.error(f"Failed to extract text from {file_name}. Skipping file.")
                                    st.error(f"❌ Failed to extract text from {file_name}")
                        else:
                            logger.warning(f"File '{file_name}' was skipped as it has already been processed.")
                            st.warning(f"File '{file_name}' has already been processed and was skipped.")

                    if needs_rerun:
                        st.rerun()
        else:
            st.warning("You have reached the maximum of 5 processed files. No more files can be added.")

            if st.session_state.processed_files:
                st.subheader("📚 Processed Documents")
                for file_name in st.session_state.processed_files:
                    st.text(f"• {file_name}")
                total_chunks = st.session_state.faiss_index.ntotal if st.session_state.faiss_index else 0
                st.caption(f"Total chunks in FAISS index: {total_chunks}")
                logger.info(f"Displaying processed files. Total chunks: {total_chunks}")

    if st.session_state.faiss_index is not None and st.session_state.faiss_index.ntotal > 0:
        st.header("💬 Ask Questions")
        
        
        st.markdown("""
        <style>
        input {
        unicode-bidi: bidi-override;
        direction: RTL;
        text-align: right !important;
        }
        div[data-testid="stTextInput"] span {
        display: none !important;
        }
        </style>
        """, unsafe_allow_html=True)
                
        query = st.text_input(
            "Enter your question (preferably in Farsi):",
            placeholder="این اسناد در مورد چیست؟"
        )

        if query:
            logger.info(f"New query received: '{query}'")
            start_time = time.perf_counter()

            processed_query = map_finglish_to_persian(query, st.session_state.finglish_map)
            if processed_query != query:
                logger.info(f"Query mapped from Finglish to: '{processed_query}'")
                st.info(f"You query has been modified automatically: «{processed_query}»")

            with st.spinner("🔍 Searching and generating answer..."):
                search_results = st.session_state.rag_system.search_documents(
                    processed_query,
                    st.session_state.text_chunks,
                    st.session_state.faiss_index,
                    top_k=TOP_K
                )

                if search_results and search_results['documents']:
                    context = "\n\n---\n\n".join(search_results['documents'][0])
                    answer = st.session_state.rag_system.generate_answer(query, context, st.session_state.chat_history)
                    answer = filter_persian_english(answer)
                    
                    if answer:
                        st.subheader("🎯 Response")
                        final_answer_body = ""
                        word_count = len(answer.split())

                        if word_count > TLDR_WORD_COUNT_THRESHOLD:
                            logger.info(f"Response is long ({word_count} words). Generating summary.")
                            with st.spinner(f"The response it too long ({word_count} words). Generating summary..."):
                                tldr_text = st.session_state.rag_system.summarize_text(answer)
                            if tldr_text:
                                final_answer_body = f"**Summary:** {tldr_text}\n\n---\n\n{answer}"
                            else:
                                logger.warning("Summary generation failed, using full answer.")
                                final_answer_body = answer
                        else:
                            final_answer_body = answer

                        end_time = time.perf_counter()
                        latency = end_time - start_time
                        logger.info(f"Total query latency: {latency:.2f} seconds.")

                        st.sidebar.info(f"Latency: {latency:.2f} seconds")
                        
                        escaped_answer = html.escape(final_answer_body).replace("\n", "<br>")
                        escaped_disclaimer = html.escape(FIXED_DISCLAIMER_FA)

                        rtl_answer_block = f"""
                        <div dir="rtl" style="text-align: right; font-family: Arial, sans-serif;">
                            {escaped_answer}
                            <br><br>
                            <hr style="border: 1px solid #ccc;">
                            <i>{escaped_disclaimer}</i>
                        </div>
                        """
                        st.markdown(rtl_answer_block, unsafe_allow_html=True)

                        st.session_state.chat_history.append({
                            "query": query,
                            "answer": final_answer_body
                        })
                        if len(st.session_state.chat_history) > 5:
                            st.session_state.chat_history.pop(0)

                        with st.expander("📖 Sources", expanded=False):

                            num_sources_to_show = min(3, len(search_results['documents'][0]))

                            for i in range(num_sources_to_show):
                                doc = search_results['documents'][0][i]
                                metadata = search_results['metadatas'][0][i]

                                st.markdown(
                                    f"<div dir='rtl' style='text-align: right;'>"
                                    f"**منبع {i + 1}:** {metadata['source']} (بخش {metadata['chunk_number']})"
                                    f"</div>",
                                    unsafe_allow_html=True
                                )
                                st.markdown(f"<div dir='rtl'>**فاصله (L2 Distance):** {metadata['score']:.4f}</div>",
                                            unsafe_allow_html=True)

                                rtl_doc_content = (
                                    f"<div dir='rtl' style='text-align: right; "
                                    f"border: 1px solid #ccc; padding: 10px; border-radius: 5px; "
                                    f"max-height: 150px; overflow-y: auto; white-space: pre-wrap;'>"
                                    f"**محتوا:** {doc}"
                                    f"</div>"
                                )
                                st.markdown(rtl_doc_content, unsafe_allow_html=True)
                                st.markdown("---")
                    else:
                        logger.error("LLM generation returned an empty answer.")
                        st.error("The model failed to generate a response.")
                else:
                    logger.warning(f"No relevant documents found for query: '{query}'")
                    st.warning("No relevant documents found for your query.")
    else:
        st.info("👆 Please upload a PDF or DOCX document using the sidebar to get started!")

Appending to app.py


In [19]:
%%writefile -a app.py

if __name__ == "__main__":
    main()

Appending to app.py


In [20]:
! streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.17.0.10:8501
  External URL: http://31.14.121.178:8501

2025-10-28 00:16:22,173 - __main__ - INFO - Application started. Running on device: cuda
2025-10-28 00:16:22,184 - __main__ - INFO - Starting main() function. Page is loading.
2025-10-28 00:16:22,187 - __main__ - INFO - Initializing 'rag_system' in session state.
2025-10-28 00:16:22,187 - SimpleRAGSystem - INFO - Initializing SimpleRAGSystem...
2025-10-28 00:16:22,203 - SimpleRAGSystem - INFO - Loading embedding model: sentence-transformers/LaBSE
2025-10-28 00:16:22,207 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/LaBSE
2025-10-28 00:16:53,929 - SimpleRAGSystem - INFO - Embedding model loaded. Dimension: 768
2025-10-28 00:16:53,936 - __main__ - INFO - Loading 'finglish_map' into session state.
2025-10-28 00:16:53,937 - __main__ - INFO - Finglish m